In [1]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import math
import numpy as np

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
graphite_water_psf = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\data_files\unwrapped\graphite_water.psf'
graphite_trajectory_dcd = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\data_files\unwrapped\output_per_20ps.dcd'

global_uni = mda.Universe(graphite_water_psf,graphite_trajectory_dcd)
Oxg_uni = global_uni.select_atoms('resname SPCE and name OW')
# Importing data

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [3]:
z_maxs = []

for frame in range(len(global_uni.trajectory)):
    global_uni.trajectory[frame]
    z_vals = []
    for atom in Oxg_uni.positions:
        z_vals.append(atom[2])
    z_maxs.append(max(z_vals))
# Appends the max z_val in each frame to z_maxs

average_z_max = sum(z_maxs)/len(z_maxs)
partition_count = 4
z_partitions = []

for i in range(int(global_uni.dimensions[2] / 10)):
    z_ceiling = i * 10
    if average_z_max < z_ceiling:
        for i in range(partition_count):
            z_partitions.append(z_ceiling / 4 * (i + 1))
        break
# Creating a slightly generalized partition, essentially just making sure our partition ends further up then out average_z_max

In [4]:

def calculate_msd(position_dict):

    part_location = position_dict['location']
    x_vals = np.array(position_dict['x_vals'])
    y_vals = np.array(position_dict['y_vals'])
    z_vals = np.array(position_dict['z_vals'])
    # Import needed data

    # Number of time points
    num_points = len(x_vals)
    
    # Initialize MSD array
    msd = np.zeros(num_points)
    
    # Calculate displacements and MSD
    for dt in range(1, num_points):
        displacements = []
        for t in range(num_points - dt):
            dx = x_vals[t + dt] - x_vals[t]
            dy = y_vals[t + dt] - y_vals[t]
            dz = z_vals[t + dt] - z_vals[t]
            displacement_squared = dx**2 + dy**2 + dz**2
            displacements.append(displacement_squared)
        
        msd[dt] = np.mean(displacements)
    
    return [msd, part_location]


In [5]:
msd_dict = {}

for atom_num in range(len(Oxg_uni)): # For each SPCE in our universe

    atom_part_movement = []
    atom_pos = {'x_vals':[],'y_vals':[],'z_vals':[]}

    for frame in range(len(global_uni.trajectory)): # For each frame in our animation
        global_uni.trajectory[frame] # Flip through each frame starting at 0 going to 500
        Oxg_positions = Oxg_uni.positions 
        current_z = Oxg_positions[atom_num][2]

        for i in range(len(z_partitions)): # For each partition we have
            if current_z < z_partitions[i]: # If our current z_value is below our z_partition[i], we label the partition in the ith partition
                if i != len(z_partitions): # If current_z was never less than z_partition[i], we know its above the partition. We then append said data and partition location
                    atom_part_movement.append(i) 
                else:
                    atom_part_movement.append(i + 1)

                atom_pos['x_vals'].append(Oxg_positions[atom_num][0])
                atom_pos['y_vals'].append(Oxg_positions[atom_num][1])
                atom_pos['z_vals'].append(current_z)
                break
                # Append position data to dictionary for the specific atom we are in & break out of loop bc we know what partition we are in

    atom_n_switch_and_pos = {}
    ticker = -1

    for i in range(len(atom_pos['z_vals'])): # Looping through each possible position point for atom 1 throughout the trajectory
        if i == 0 or atom_part_movement[i - 1] != atom_part_movement[i]:
            ticker += 1
            atom_n_switch_and_pos[ticker] = {'location':atom_part_movement[i],'x_vals':[],'y_vals':[],'z_vals':[]}
        # If we havent created a struct yet or our current part changed from the last part, create a struct

        atom_n_switch_and_pos[ticker]['x_vals'].append(atom_pos['x_vals'][i])
        atom_n_switch_and_pos[ticker]['y_vals'].append(atom_pos['y_vals'][i])
        atom_n_switch_and_pos[ticker]['z_vals'].append(atom_pos['z_vals'][i])
        # Appends positions to correct structure location


    msd_dict[atom_num] = {}
    for i in range(len(z_partitions)):
        msd_dict[atom_num][i] = []

    for key in atom_n_switch_and_pos.keys(): # For each partition switch n atom did
        msd_array, msd_current_part = calculate_msd(atom_n_switch_and_pos[key])
        msd_dict[atom_num][msd_current_part].append(msd_array)
    # Stores MSD data in msd_dict for each atom & each partition shift

#{ atom_number:{ a:[ [msd_0, msd_1, msd_2, ... ], [msd_0, msd_1, msd_2, ... ], ... ], b:[ ... ], ... }, ... }

In [6]:
partition_msd = {}
for i in range(len(z_partitions)):
    partition_msd[i] = []
# Creating following structure -> {0:[],1:[],...n:[]}

for atom in msd_dict.keys():
    for i in range(len(z_partitions)):
        partition_msd[i] = partition_msd[i] + msd_dict[atom][i]
# Appending each elements list to partition_msd

In [7]:
structured_msd = {}

for key in partition_msd.keys():
    structured_msd[key] = {}
    for item in partition_msd[key]:
        if len(item) in structured_msd[key].keys():
            structured_msd[key][len(item)].append(item)
        else:
            structured_msd[key][len(item)] = [item]